Klasse empamos_rest_client

Dient dazu die EMPAMOS-REST Schnittstelle abzufragen

get_pattern(pattern_id):

    Liefert JSON mit empirischen Evidenzen zu Pattern <pattern_id>
    
    Vgl.: https://empamos.in.th-nuernberg.de/api/patterns/gesellschaftsspiele/1473?uname=empamos_dev&upass=7596512694984216578462165792221485548713215
    
get_patterns():

    Liefert JSON mit allen Patterns. (ID + Name)
    
    Vgl.: https://empamos.in.th-nuernberg.de/api/patterns/gesellschaftsspiele?uname=empamos_dev&upass=7596512694984216578462165792221485548713215
    
    
get_patterns_name_dictionary():

    Liefert ein Dictionary mit key = Pattern ID, value = Name des Patterns
    
get_negative_patterns():

    Wie get_patterns() aber mit Negative Patterns
    
    Vgl.: https://empamos.in.th-nuernberg.de/api/negativepatterns/gesellschaftsspiele?uname=empamos_dev&upass=7596512694984216578462165792221485548713215
    
get_pattern_evidence_words(pattern_id):

    Liefert eine Liste von Wörtern (lower case, einzigartig). Wörter sind manuell gefundene Evidenzen
    
get_pattern_evidence_sentences(pattern_id):

    Liefert eine Liste von Sätzen (ggf. mit Dupletten). Sätze entahelten manuell gefundene Evidenzen
    
get_pattern_evidence_paragraphs(pattern_id):

    Wie get_pattern_evidence_sentences() aber mit Paragraphen
    
evidence_pattern_search(hasAllPatterns, hasNotPatterns, hasAllNegativePatterns, hasNotNegativePatterns):

    Sucht Evidenzen (Spiele), die alle hasAllPatterns, aber keine hasNotPatterns als Pattern haben und gleichzeitig alle hasAllNegativePatterns, aber keine hasNotNegativePatterns als NegativePatterns haben.
    
    Die Parameter können als Integer, String, oder als List von Integerns/Strings übergeben werden. Ein Integer <= 0, der leere String "" oder eine leere Liste bedeuten dabei, dass der Parameter bei der Abfrage nicht berücksichtigt werden soll.
    
    Vgl.: https://empamos.in.th-nuernberg.de/api/evidences/gesellschaftsspiele?hasallpatterns=8528&hasnotpatterns=2894&hasallnegativepatterns=1292&hasnotnegativepatterns=164466&uname=xxx&upass=yyy


get_all_pattern_evidence_words():

    Liefert Dictionary mit Wörtern von allen Patterns.
    Schema: key = pattern ID; value = Liste von allen zugehörigen Wörtern

In [1]:
import requests

In [2]:
class empamos_rest_client:
    
    def __init__ (self, *args, **kwargs):
        self.baseURL = "https://empamos.in.th-nuernberg.de/api/"
        self.credentials = {'uname': 'empamos_dev', 'upass': '7596512694984216578462165792221485548713215'}

    
    def get_pattern(self, pattern_id):
        query = "patterns/gesellschaftsspiele/"
        request_data = requests.get(self.baseURL + query + str(pattern_id), params=self.credentials)
        return request_data.json()
    
    def get_patterns(self):
        query = "patterns/gesellschaftsspiele"
        request_data = requests.get(self.baseURL + query, params=self.credentials)
        return request_data.json()
    
    def get_patterns_name_dictionary(self):
        all_patterns = self.get_patterns()
        dict = {}
        for pat in all_patterns:
            if "pattern_id" in pat and "pattern_name" in pat:
                dict[pat["pattern_id"]] = pat["pattern_name"]
        return dict
    
    def get_negative_patterns(self):
        query = "negativepatterns/gesellschaftsspiele"
        request_data = requests.get(self.baseURL + query, params=self.credentials)
        return request_data.json()
    
    def get_pattern_evidence_words(self, pattern_id):
        words = []
        pattern_data = self.get_pattern(pattern_id)
        for evidence in pattern_data["empiricalevidence"]:
            if "paragraphs" in evidence:
                for paragraph in evidence["paragraphs"]:
                    if "sentences" in paragraph:
                        for sentence in paragraph["sentences"]:
                            if "words" in sentence:
                                for word in sentence["words"]:
                                    if not word["word_text"].lower() in words:
                                        words.append(word["word_text"].lower())
        return words
    
    def get_pattern_evidence_sentences(self, pattern_id):
        sentences = []
        pattern_data = self.get_pattern(pattern_id)
        if "empiricalevidence" in pattern_data:
            for evidence in pattern_data["empiricalevidence"]:
                if "paragraphs" in evidence:
                    for paragraph in evidence["paragraphs"]:
                        if "sentences" in paragraph:
                            for sentence in paragraph["sentences"]:
                                if "sentence_text" in sentence:
                                    sentences.append(sentence["sentence_text"])
        return sentences
    
    def get_pattern_evidence_paragraphs(self, pattern_id):
        paragraphs = []
        pattern_data = self.ge_pattern(pattern_id)
        if "empiricalevidence" in pattern_data:
            for evidence in pattern_data["empiricalevidence"]:
                if "paragraphs" in evidence:
                    for paragraph in evidence["paragraphs"]:
                        if "paragraph_text" in paragraph:
                            paragraphs.append(paragraph["paragraph_text"])
        return paragraphs
    
    def get_all_pattern_evidence_words(self):
        patternTerms = {}
        patterns = self.get_patterns();
        for pat in patterns:
            if "pattern_id" in pat:
                patternTerms[pat["pattern_id"]] = self.get_pattern_evidence_words(pat["pattern_id"]);
        return patternTerms


    
    #parameter sind entweder Integer / Strings oder Listen von Integer / Strings. <=0 / "" Heißt leer lassen
    def evidence_pattern_search(self, hasAllPatterns, hasNotPatterns, hasAllNegativePatterns, hasNotNegativePatterns):
        params = {}
        self.add_param(hasAllPatterns,params,"hasallpatterns")
        self.add_param(hasNotPatterns,params,"hasnotpatterns")
        self.add_param(hasAllNegativePatterns,params,"hasallnegativepatterns")
        self.add_param(hasNotNegativePatterns,params,"hasnotnegativenatterns")
        query = "evidences/gesellschaftsspiele"
        request_data = requests.get(self.baseURL + query, params={**self.credentials,**params})
        return request_data.json()
            
    def add_param(self, param, param_dict, name):
        if isinstance(param, int):
            if param > 0:
                param_dict[name] = param
        elif isinstance(param, str):
            if len(param) > 0 :
                param_dict[name] = param
        else:
            if len(param) > 0:
                param_dict[name] = ",".join(str(p) for p in param)
        

client = empamos_rest_client()
p = client.get_patterns_name_dictionary()

print(client.get_pattern(183168))
pattern = 1473
print("patterns")
print(client.get_patterns())
print("")
print("negative patterns")
print(client.get_negative_patterns())
print("")
print("words for " + str(pattern))
print(client.get_pattern_evidence_words(pattern))
print("")
print("sentences for " + str(pattern))
print(client.get_pattern_evidence_sentences(pattern))

print("")
print("Random evidence pattern search")
print(client.evidence_pattern_search([8528,1473],2894,"1292",""))

client = empamos_rest_client()

print(client.get_pattern_evidence_words(167602))